<a href="https://colab.research.google.com/github/scottthomaswx/Nadocast_Zoom/blob/main/Nadocast_Zoom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Needed Packages

In [1]:
# @title
%%capture
!pip install xarray
!pip install xarray['complete']
!pip install cfgrib
!pip install cfgrib xarray
!apt-get install -y libeccodes0
!pip install eccodes
!pip install matplotlib
!pip install metpy
!pip install warnings

import xarray as xr
import requests
from metpy.plots import USCOUNTIES
import cartopy.feature as cfeat
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from metpy.plots import MapPanel, PanelContainer, PlotGeometry
import warnings

!rm -rf sample_data

#Set Nadocast Runtime

## Leading Zeroes Required In Day/Month

### Period 1 = Day 1 (Available For All Runs)
### Period 2 = Day 2 (Available Only at 12z)

##Only Tornado Data and No Significant Hazard Data Available At 14z


In [21]:
year = '2025' #@param {type:"string"}
month = '05' #@param {type:"string"}
day = '15' #@param {type:"string"}
time = '12' #@param {type:"string"}
period = '1' #@param {type:"string"}

#Product Options

### For 14z Runs You Must Use tornado

In [22]:
product = 'all_hazards' #@param ['hail', 'wind', 'tornado', 'sig_hail', 'sig_wind', 'sig_tornado', 'wind_all', 'hail_all', 'tornado_all', 'wind_all', 'all_hazards']
type_spcabs = 'SPC Like' #@param ['SPC Like', 'Absolutey Calibrated']

# Set Map Boundaries

In [23]:
WestLong = -90 # @param {type:"number"}
EastLong = -82 # @param {type:"number"}
NorthLat = 45 # @param {type:"number"}
SouthLat = 40 # @param {type:"number"}

#Pull Nadocast

In [ ]:
# @title
if type_spcabs == 'SPC Like':
  if product == 'tornado' or product == 'wind' or product == 'hail':
    if period == '1':
      if time == '00' or time == '0':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f12-35.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f12-35.grib2'
      elif time == '12':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f02-23.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f02-23.grib2'
      elif time == '14':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2020_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f02-21.grib2'
      elif time == '18':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f02-17.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f02-17.grib2'
      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f24-47.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')

    r1 = requests.get(url)
    file1 = 'nadocast.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

    r2 = requests.get(sigurl)
    file2 = 'nadocast_sig.grb2'
    with open(file2, 'wb') as f2:
      f2.write(r2.content)

  elif  product == 'sig_tornado' or product == 'sig_hail' or product == 'sig_wind':
    if period == '1':
      if time == '00' or time == '0':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f12-35.grib2'
      elif time == '12':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f02-23.grib2'
      elif time == '18':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f02-17.grib2'
      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')
    r1 = requests.get(url)
    file1 = 'nadocast.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

  elif product == 'hail_all' or product == 'wind_all' or product == 'tornado_all':
    if product == 'hail_all':
      subproduct = 'hail'
    elif product == 'wind_all':
      subproduct = 'wind'
    elif product == 'tornado_all':
      subproduct = 'tornado'

    if period == '1':
      if time == '00' or time == '0':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_{subproduct}_{year}{month}{day}_t{time}z_f12-35.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_{subproduct}_{year}{month}{day}_t{time}z_f12-35.grib2'
      elif time == '12':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{subproduct}_{year}{month}{day}_t{time}z_f02-23.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{subproduct}_{year}{month}{day}_t{time}z_f02-23.grib2'
      elif time == '18':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{subproduct}_{year}{month}{day}_t{time}z_f02-17.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{subproduct}_{year}{month}{day}_t{time}z_f02-17.grib2'
      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{subproduct}_{year}{month}{day}_t{time}z_f24-47.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{subproduct}_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')

    r1 = requests.get(url)
    file1 = 'nadocast.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

    r2 = requests.get(sigurl)
    file2 = 'nadocast_sig.grb2'
    with open(file2, 'wb') as f2:
      f2.write(r2.content)

  elif product == 'all_hazards':
    if period == '1':
      if time == '00' or time == '0':
          torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_tornado_{year}{month}{day}_t{time}z_f12-35.grib2'
          sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_tornado_{year}{month}{day}_t{time}z_f12-35.grib2'
          hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_hail_{year}{month}{day}_t{time}z_f12-35.grib2'
          sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_hail_{year}{month}{day}_t{time}z_f12-35.grib2'
          windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_wind_{year}{month}{day}_t{time}z_f12-35.grib2'
          sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_wind_{year}{month}{day}_t{time}z_f12-35.grib2'

      elif time == '12':
        torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_tornado_{year}{month}{day}_t{time}z_f02-23.grib2'
        sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_tornado_{year}{month}{day}_t{time}z_f02-23.grib2'
        hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_hail_{year}{month}{day}_t{time}z_f02-23.grib2'
        sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_hail_{year}{month}{day}_t{time}z_f02-23.grib2'
        windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_wind_{year}{month}{day}_t{time}z_f02-23.grib2'
        sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_wind_{year}{month}{day}_t{time}z_f02-23.grib2'

      elif time == '18':
        torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_tornado_{year}{month}{day}_t{time}z_f02-17.grib2'
        sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_tornado_{year}{month}{day}_t{time}z_f02-17.grib2'
        hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_hail_{year}{month}{day}_t{time}z_f02-17.grib2'
        sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_hail_{year}{month}{day}_t{time}z_f02-17.grib2'
        windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_wind_{year}{month}{day}_t{time}z_f02-17.grib2'
        sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_wind_{year}{month}{day}_t{time}z_f02-17.grib2'

      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_tornado_{year}{month}{day}_t{time}z_f24-47.grib2'
        sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_tornado_{year}{month}{day}_t{time}z_f24-47.grib2'
        hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_hail_{year}{month}{day}_t{time}z_f24-47.grib2'
        sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_hail_{year}{month}{day}_t{time}z_f24-47.grib2'
        windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_wind_{year}{month}{day}_t{time}z_f24-47.grib2'
        sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_wind_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')

    r1 = requests.get(torurl)
    file1 = 'tornado.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

    r2 = requests.get(sigtorurl)
    file2 = 'tornado_sig.grb2'
    with open(file2, 'wb') as f2:
      f2.write(r2.content)

    r3 = requests.get(hailurl)
    file3 = 'hail.grb2'
    with open(file3, 'wb') as f3:
      f3.write(r3.content)

      r4 = requests.get(sighailurl)
    file4 = 'hail_sig.grb2'
    with open(file4, 'wb') as f4:
      f4.write(r4.content)

    r5 = requests.get(windurl)
    file5 = 'wind.grb2'
    with open(file5, 'wb') as f5:
      f5.write(r5.content)

    r6 = requests.get(sigwindurl)
    file6 = 'wind_sig.grb2'
    with open(file6, 'wb') as f6:
      f6.write(r6.content)

elif type_spcabs == 'Absolutey Calibrated':
  if product == 'tornado' or product == 'wind' or product == 'hail':
    if period == '1':
      if time == '00' or time == '0':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
      elif time == '12':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
      elif time == '14':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2020_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-21.grib2'
      elif time == '18':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')

    r1 = requests.get(url)
    file1 = 'nadocast.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

    r2 = requests.get(sigurl)
    file2 = 'nadocast_sig.grb2'
    with open(file2, 'wb') as f2:
      f2.write(r2.content)

  elif  product == 'sig_tornado' or product == 'sig_hail' or product == 'sig_wind':
    if period == '1':
      if time == '00' or time == '0':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
      elif time == '12':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
      elif time == '18':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')
    r1 = requests.get(url)
    file1 = 'nadocast.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

  elif product == 'hail_all' or product == 'wind_all' or product == 'tornado_all':
    if product == 'hail_all':
      subproduct = 'hail'
    elif product == 'wind_all':
      subproduct = 'wind'
    elif product == 'tornado_all':
      subproduct = 'tornado'

    if period == '1':
      if time == '00' or time == '0':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_{subproduct}_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_{subproduct}_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
      elif time == '12':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{subproduct}_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{subproduct}_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
      elif time == '18':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{subproduct}_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{subproduct}_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{subproduct}_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{subproduct}_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')

    r1 = requests.get(url)
    file1 = 'nadocast.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

    r2 = requests.get(sigurl)
    file2 = 'nadocast_sig.grb2'
    with open(file2, 'wb') as f2:
      f2.write(r2.content)

  elif product == 'all_hazard':
    if period == '1':
      if time == '00' or time == '0':
          torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_tornado_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
          sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_tornado_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
          hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_hail_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
          sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_hail_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
          windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_wind_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
          sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_wind_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'

      elif time == '12':
        torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_tornado_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
        sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_tornado_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
        hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_hail_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
        sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_hail_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
        windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_wind_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
        sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_wind_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'

      elif time == '18':
        torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_tornado_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
        sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_tornado_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
        hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_hail_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
        sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_hail_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
        windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_wind_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
        sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_wind_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_tornado_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
        sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_tornado_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
        hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_hail_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
        sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_hail_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
        windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_wind_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
        sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_wind_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')

    r1 = requests.get(torurl)
    file1 = 'tornado.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

    r2 = requests.get(sigtorurl)
    file2 = 'tornado_sig.grb2'
    with open(file2, 'wb') as f2:
      f2.write(r2.content)

    r3 = requests.get(hailurl)
    file3 = 'hail.grb2'
    with open(file3, 'wb') as f3:
      f3.write(r3.content)

    r4 = requests.get(sighailurl)
    file4 = 'hail_sig.grb2'
    with open(file4, 'wb') as f4:
      f4.write(r4.content)

    r5 = requests.get(windurl)
    file5 = 'wind.grb2'
    with open(file5, 'wb') as f5:
      f5.write(r5.content)

    r6 = requests.get(sigwindurl)
    file6 = 'wind_sig.grb2'
    with open(file6, 'wb') as f6:
      f6.write(r6.content)

else:
  raise ValueError('Invalid Product Selected')

if product == 'tornado' or product == 'wind' or product == 'hail' or product == 'sig_tornado' or product == 'sig_hail' or product == 'sig_wind':
  try:
    xr.open_mfdataset('/content/nadocast.grb2', engine="cfgrib")
  except:
    raise ValueError('Invalid Product Selected or Product Not Available')

  try:
    xr.open_mfdataset('/content/nadocast_sig.grb2', engine="cfgrib")
  except:
    raise ValueError('Invalid Product Selected or Product Not Available')

if (product == 'hail_all' or product == 'wind_all' or product == 'tornado_all') and (time != '14'):
  try:
    xr.open_mfdataset('/content/nadocast.grb2', engine="cfgrib")
  except:
    raise ValueError('Invalid Product Selected or Product Not Available')

  try:
    xr.open_mfdataset('/content/nadocast_sig.grb2', engine="cfgrib")
  except:
    raise ValueError('Invalid Product Selected or Product Not Available')

if (product == 'all_hazards') and (time != '14'):
  try:
    xr.open_mfdataset('/content/tornado.grb2', engine="cfgrib")
  except:
    raise ValueError('Tornado Forecast Unavailable... Please Attempt Using Single Hazard or Individual Parameter Options')

  try:
    xr.open_mfdataset('/content/tornado_sig.grb2', engine="cfgrib")
  except:
    raise ValueError('Significant Tornado Forecast Unavailable... Please Attempt Using Single Hazard or Individual Parameter Options')

  try:
    xr.open_mfdataset('/content/hail.grb2', engine="cfgrib")
  except:
    raise ValueError('Hail Forecast Unavailable... Please Attempt Using Single Hazard or Individual Parameter Options')

  try:
    xr.open_mfdataset('/content/hail_sig.grb2', engine="cfgrib")
  except:
    raise ValueError('Significant Hail Forecast Unavailable... Please Attempt Using Single Hazard or Individual Parameter Options')

  try:
    xr.open_mfdataset('/content/wind.grb2', engine="cfgrib")
  except:
    raise ValueError('Wind Forecast Unavailable... Please Attempt Using Single Hazard or Individual Parameter Options')

  try:
    xr.open_mfdataset('/content/wind_sig.grb2', engine="cfgrib")
  except:
    raise ValueError('Significant Wind Unavailable... Please Attempt Using Single Hazard or Individual Parameter Options')

#Open Nadocast File

In [ ]:
# @title
%%capture
if product != 'all_hazards':
  nadocastfile = xr.open_mfdataset('/content/nadocast.grb2', engine="cfgrib")

if product == 'tornado' or product == 'wind' or product == 'hail' or product == 'tornado_all' or product == 'wind_all' or product == 'hail_all':
  signadocastfile = xr.open_mfdataset('/content/nadocast_sig.grb2', engine="cfgrib")

if product == 'tornado':
  prob = nadocastfile.torprob
  product_label = "Tornado"
  sigprob = signadocastfile.storprob

  levels = [0.1,1,2,3,5,10,15,30,45,60, 100]
  color_list = ['#f4f4f4','#dcdcdc','#118a14','#53cf55','#8a472a','#fec72e','#fc0d1b','#fc28fc','#9039ea','#154f89']

  siglevel = [10,100]
  sig_hatch = ['////']
  sig_color = ['none']

  PN1 = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#f4f4f4')
  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#dcdcdc')
  TWO = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#118a14')
  THR = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#53cf55')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  TEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#154f89')
  SIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')

  legend_patches = [PN1, ONE, TWO, THR, FIV, TEN, FTN, TTY, FFV, SXY, SIG]
  legend_labels = ['0.1%','1%','2%','3%','5%','10%','15%','30%','45%','60%', 'SIG']

elif product == 'hail':
  prob = nadocastfile.hailprob
  product_label = "Hail"
  sigprob = signadocastfile.shailpro

  levels = [1,5,15,30,45,60, 100]
  color_list = ['#e6e6e6', '#8a472a', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']
  siglevel = [10,100]
  sig_hatch = ['////']
  sig_color = ['none']

  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  SIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')

  legend_patches = [ONE, FIV, FTN, TTY, FFV, SXY, SIG]
  legend_labels = ['1%','5%','15%','30%','45%','60%', 'SIG']

elif product == 'wind':
  prob = nadocastfile.windprob
  product_label = "Wind"
  sigprob = signadocastfile.swindpro

  levels = [1,5,15,30,45,60, 100]
  color_list = ['#e6e6e6', '#8a472a', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']
  siglevel = [10,100]
  sig_hatch = ['////']
  sig_color = ['none']

  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  SIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')

  legend_patches = [ONE, FIV, FTN, TTY, FFV, SXY, SIG]
  legend_labels = ['1%','5%','15%','30%','45%','60%','SIG']

elif product == 'sig_tornado':
  prob = nadocastfile.storprob
  product_label = "Significant Tornado"

  levels = [0.1,1,2,3,5,10,15,30,45,60, 100]
  color_list = ['#f4f4f4','#dcdcdc','#118a14','#53cf55','#8a472a','#4d4d4d','#fc0d1b','#fc28fc','#9039ea','#154f89']

  PN1 = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#f4f4f4')
  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#dcdcdc')
  TWO = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#118a14')
  THR = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#53cf55')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  TEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#154f89')

  legend_patches = [PN1, ONE, TWO, THR, FIV, TEN, FTN, TTY, FFV, SXY]
  legend_labels = ['0.1%','1%','2%','3%','5%','10%','15%','30%','45%','60%']

elif product == 'sig_hail':
  prob = nadocastfile.shailpro
  product_label = "Significant Hail"

  levels = [1,5,10,15,30,45,60, 100]
  color_list = ['#e6e6e6', '#8a472a', '#4d4d4d', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']

  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  TEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')

  legend_patches = [ONE, FIV, TEN, FTN, TTY, FFV, SXY]
  legend_labels = ['1%','5%','10%', '15%','30%','45%','60%']

elif product == 'sig_wind':
  prob = nadocastfile.swindpro
  product_label = "Significant Wind"

  levels = [1,5,10,15,30,45,60, 100]
  color_list = ['#e6e6e6', '#8a472a', '#4d4d4d', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']


  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  TEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')

  legend_patches = [ONE, FIV, TEN, FTN, TTY, FFV, SXY]
  legend_labels = ['1%','5%','10%', '15%','30%','45%','60%']

if product == 'tornado_all':
  prob = nadocastfile.torprob
  product_label = "Tornado"
  sigprob = signadocastfile.storprob

  problevels = [0.1,1,2,3,5,10,15,30,45,60, 100]
  probcolor_list = ['#f4f4f4','#dcdcdc','#118a14','#53cf55','#8a472a','#fec72e','#fc0d1b','#fc28fc','#9039ea','#154f89']

  probsiglevel = [10,100]
  probsig_hatch = ['////']
  probsig_color = ['none']

  PN1 = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#f4f4f4')
  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#dcdcdc')
  TWO = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#118a14')
  THR = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#53cf55')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  TEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#154f89')
  SIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')

  problegend_patches = [PN1, ONE, TWO, THR, FIV, TEN, FTN, TTY, FFV, SXY, SIG]
  problegend_labels = ['0.1%','1%','2%','3%','5%','10%','15%','30%','45%','60%', 'SIG']

  sigproduct_label = "Significant Tornado"

  siglevels = [0.1,1,2,3,5,10,15,30,45,60, 100]
  sigcolor_list = ['#f4f4f4','#dcdcdc','#118a14','#53cf55','#8a472a','#4d4d4d','#fc0d1b','#fc28fc','#9039ea','#154f89']

  SPN1 = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#f4f4f4')
  SONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#dcdcdc')
  STWO = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#118a14')
  STHR = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#53cf55')
  SFIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  STEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  SFTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  STTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  SFFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  SSXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#154f89')

  slegend_patches = [SPN1, SONE, STWO, STHR, SFIV, STEN, SFTN, STTY, SFFV, SSXY]
  slegend_labels = ['0.1%','1%','2%','3%','5%','10%','15%','30%','45%','60%']

elif product == 'hail_all':
  prob = nadocastfile.hailprob
  product_label = "Hail"
  sigprob = signadocastfile.shailpro

  problevels = [1,5,10,15,30,45,60, 100]
  probcolor_list = ['#e6e6e6', '#8a472a', '#4d4d4d', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']
  probsiglevel = [10,100]
  probsig_hatch = ['////']
  probsig_color = ['none']

  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  TEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  SIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')
  problegend_patches = [ONE, FIV, TEN, FTN, TTY, FFV, SXY, SIG]
  problegend_labels = ['1%','5%','10%', '15%','30%','45%','60%', 'SIG']

  siglevels = [1,5,10,15,30,45,60, 100]
  sigcolor_list = ['#e6e6e6', '#8a472a', '#4d4d4d', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']

  SONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  SFIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  STEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  SFTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  STTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  SFFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  SSXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')

  Slegend_patches = [SONE, SFIV, STEN, SFTN, STTY, SFFV, SSXY]
  Slegend_labels = ['1%','5%','10%', '15%','30%','45%','60%']

elif product == 'wind_all':
  prob = nadocastfile.windprob
  product_label = "Wind"
  sigprob = signadocastfile.swindpro

  problevels = [1,5,10,15,30,45,60, 100]
  probcolor_list = ['#e6e6e6', '#8a472a', '#4d4d4d', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']
  probsiglevel = [10,100]
  probsig_hatch = ['////']
  probsig_color = ['none']

  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  TEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  SIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')
  problegend_patches = [ONE, FIV, TEN, FTN, TTY, FFV, SXY, SIG]
  problegend_labels = ['1%','5%','10%', '15%','30%','45%','60%', 'SIG']

  siglevels = [1,5,10,15,30,45,60, 100]
  sigcolor_list = ['#e6e6e6', '#8a472a', '#4d4d4d', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']

  SONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  SFIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  STEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  SFTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  STTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  SFFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  SSXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')

  Slegend_patches = [SONE, SFIV, STEN, SFTN, STTY, SFFV, SSXY]
  Slegend_labels = ['1%','5%','10%', '15%','30%','45%','60%']

if product == 'all_hazards' and time != '14':

  torfile = xr.open_mfdataset('/content/tornado.grb2')
  hailfile = xr.open_mfdataset('/content/hail.grb2')
  windfile = xr.open_mfdataset('/content/wind.grb2')
  storfile = xr.open_mfdataset('/content/tornado_sig.grb2')
  shailfile = xr.open_mfdataset('/content/hail_sig.grb2')
  swindfile = xr.open_mfdataset('/content/wind_sig.grb2')

  torprob = torfile.torprob
  hailprob = hailfile.hailprob
  windprob = windfile.windprob
  storprob = storfile.storprob
  shailprob = shailfile.shailpro
  swindprob = swindfile.swindpro

  tproblevels = [0.1,1,2,3,5,10,15,30,45,60, 100]
  tprobcolor_list = ['#f4f4f4','#dcdcdc','#118a14','#53cf55','#8a472a','#fec72e','#fc0d1b','#fc28fc','#9039ea','#154f89']

  tprobsiglevel = [10,100]
  tprobsig_hatch = ['////']
  tprobsig_color = ['none']

  tPN1 = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#f4f4f4')
  tONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#dcdcdc')
  tTWO = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#118a14')
  tTHR = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#53cf55')
  tFIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  tTEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  tFTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  tTTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  tFFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  tSXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#154f89')
  tSIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')

  tproblegend_patches = [tPN1, tONE, tTWO, tTHR, tFIV, tTEN, tFTN, tTTY, tFFV, tSXY, tSIG]
  tproblegend_labels = ['0.1%','1%','2%','3%','5%','10%','15%','30%','45%','60%', 'SIG']

  tsigproduct_label = "Significant Tornado"

  tsiglevels = [0.1,1,2,3,5,10,15,30,45,60, 100]
  tsigcolor_list = ['#f4f4f4','#dcdcdc','#118a14','#53cf55','#8a472a','#4d4d4d','#fc0d1b','#fc28fc','#9039ea','#154f89']

  StPN1 = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#f4f4f4')
  StONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#dcdcdc')
  StTWO = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#118a14')
  StTHR = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#53cf55')
  StFIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  StTEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  StFTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  StTTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  StFFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  StSXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#154f89')

  stlegend_patches = [StPN1, StONE, StTWO, StTHR, StFIV, StTEN, StFTN, StTTY, StFFV, StSXY]
  stlegend_labels = ['0.1%','1%','2%','3%','5%','10%','15%','30%','45%','60%']

  hproblevels = [1,5,10,15,30,45,60, 100]
  hprobcolor_list = ['#e6e6e6', '#8a472a', '#4d4d4d', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']
  hprobsiglevel = [10,100]
  hprobsig_hatch = ['////']
  hprobsig_color = ['none']

  hONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  hFIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  hTEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  hFTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  hTTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  hFFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  hSXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  hSIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')
  hproblegend_patches = [hONE, hFIV, hTEN, hFTN, hTTY, hFFV, hSXY, hSIG]
  hproblegend_labels = ['1%','5%','10%', '15%','30%','45%','60%', 'SIG']

  hsiglevels = [1,5,10,15,30,45,60, 100]
  hsigcolor_list = ['#e6e6e6', '#8a472a', '#4d4d4d', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']

  hSONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  hSFIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  hSTEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  hSFTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  hSTTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  hSFFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  hSSXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')

  hSlegend_patches = [hSONE, hSFIV, hSTEN, hSFTN, hSTTY, hSFFV, hSSXY]
  hSlegend_labels = ['1%','5%','10%', '15%','30%','45%','60%']

  wproblevels = [1,5,10,15,30,45,60, 100]
  wprobcolor_list = ['#e6e6e6', '#8a472a', '#4d4d4d', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']
  wprobsiglevel = [10,100]
  wprobsig_hatch = ['////']
  wprobsig_color = ['none']

  wONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  wFIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  wTEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  wFTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  wTTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  hFFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  wSXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  wSIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')
  wproblegend_patches = [hONE, hFIV, hTEN, hFTN, hTTY, hFFV, hSXY, hSIG]
  wproblegend_labels = ['1%','5%','10%', '15%','30%','45%','60%', 'SIG']

  wsiglevels = [1,5,10,15,30,45,60, 100]
  wsigcolor_list = ['#e6e6e6', '#8a472a', '#4d4d4d', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']

  wSONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  wSFIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  wSTEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  wSFTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  wSTTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  wSFFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  wSSXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')

  wSlegend_patches = [hSONE, hSFIV, hSTEN, hSFTN, hSTTY, hSFFV, hSSXY]
  wSlegend_labels = ['1%','5%','10%', '15%','30%','45%','60%']

#Create Image

In [ ]:
# @title
#%%capture
if product == 'hail' or product == 'tornado' or product == 'wind' or product == 'sig_hail' or product == 'sig_tornado' or product == 'sig_wind':
  #Set Up Figure
  fig = plt.figure(figsize=(12, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
  ax.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax.set_box_aspect(0.7)
  box1 = ax.get_position()
  ax.set_position([box1.x0, box1.y0, box1.width * 0.8, box1.height*0.95])

  #Plot Data
  nado_contour = ax.contourf(prob.longitude, prob.latitude, prob.values, levels = levels, colors = color_list)
  if product == 'tornado' or product == 'hail' or product == 'wind':
    sig_contour = ax.contourf(sigprob.longitude, sigprob.latitude, sigprob.values, levels = siglevel, colors = sig_color, hatches = sig_hatch)
    sig_line = ax.contour(sigprob.longitude, sigprob.latitude, sigprob.values, levels = siglevel, colors = 'black', linewidths = 1)

  #Create Legend
  ax.legend(legend_patches, legend_labels,  loc='upper left', fontsize=12, facecolor='white', markerscale=12,
                labelcolor='k', title='  Risk', title_fontsize=12, edgecolor='black', framealpha=1,bbox_to_anchor=(1.10, 1), fancybox=True, shadow=True,)

  #Add Counties and Coastlines
  ax.coastlines(color='black',resolution='10m',zorder=99)
  ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='darkgray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax.add_feature(lake)

  #Add Figure Text
  plt.figtext(0.11, 0.23, f'SOURCE:Nadocast', weight='bold', color='black', fontsize=13, ha='left', alpha=0.8)

  if type_spcabs == 'SPC Like':
    if period == '1':
      plt.figtext(0.45, 0.73, f'Nadocast Day 1 {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)
    if period == '2':
      plt.figtext(0.45, 0.73, f'Nadocast Day 2 {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)
  elif type_spcabs == 'Absolutey Calibrated':
    if period == '1':
      plt.figtext(0.45, 0.73, f'Nadocast Day 1 Absolutely Calibrated {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)
    if period == '2':
      plt.figtext(0.45, 0.73, f'Nadocast Day 2 Absolutely Calibrated {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)

  #Add Gridlines
  gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl.x_labels_top = False
  gl.x_labels_bottom = True

  #Save Image
  if type_spcabs == 'SPC Like':
    plt.savefig(f'Nadocast_Day{period}_{product}_{year}-{month}-{day}-{time}UTC.png', bbox_inches = 'tight')
  elif type_spcabs == 'Absolutey Calibrated':
    plt.savefig(f'Nadocast_Day{period}_{product}_Abs_Calib_{year}-{month}-{day}-{time}UTC.png', bbox_inches = 'tight')

elif (product == 'tornado_all' or 'wind_all' or 'hail_all') and (time != '14') and (product !='all_hazards'):
  #Set Up Figure
  fig = plt.figure(figsize=(16, 12), facecolor='white', edgecolor="black", linewidth =6)


  ax=fig.add_subplot(1,2,1, projection=ccrs.PlateCarree())
  ax.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax.set_box_aspect(0.7)
  box1 = ax.get_position()
  ax.set_position([box1.x0, box1.y0, box1.width * 0.8, box1.height*0.95])

  #Plot Data
  nado_contour = ax.contourf(prob.longitude, prob.latitude, prob.values, levels = levels, colors = color_list)
  if product == 'tornado' or product == 'hail' or product == 'wind':
    sig_contour = ax.contourf(sigprob.longitude, sigprob.latitude, sigprob.values, levels = siglevel, colors = sig_color, hatches = sig_hatch)
    sig_line = ax.contour(sigprob.longitude, sigprob.latitude, sigprob.values, levels = siglevel, colors = 'black', linewidths = 1)

  #Create Legend
  ax.legend(legend_patches, legend_labels,  loc='upper left', fontsize=12, facecolor='white', markerscale=12,
                labelcolor='k', title='  Risk', title_fontsize=12, edgecolor='black', framealpha=1,bbox_to_anchor=(1.10, 1), fancybox=True, shadow=True,)

  #Add Counties and Coastlines
  ax.coastlines(color='black',resolution='10m',zorder=99)
  ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='darkgray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax.add_feature(lake)

  #Add Figure Text
  plt.figtext(0.11, 0.32, f'SOURCE:Nadocast', weight='bold', color='black', fontsize=13, ha='left', alpha=0.8)

  if type_spcabs == 'SPC Like':
    if period == '1':
      plt.figtext(0.5, 0.68, f'Nadocast Day 1 {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)
    if period == '2':
      plt.figtext(0.5, 0.68, f'Nadocast Day 2 {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)
  elif type_spcabs == 'Absolutey Calibrated':
    if period == '1':
      plt.figtext(0.5, 0.68, f'Nadocast Day 1 Absolutely Calibrated {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)
    if period == '2':
      plt.figtext(0.5, 0.68, f'Nadocast Day 2 Absolutely Calibrated {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)

  plt.figtext(0.21, 0.65, f'Overall {product_label} Risk', ha='left', weight='bold', fontsize=16)
  plt.figtext(0.61, 0.65, f'Significant {product_label} Risk', ha='left', weight='bold', fontsize=16)


  ax2=fig.add_subplot(1,2,2, projection=ccrs.PlateCarree())
  ax2.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax2.set_box_aspect(0.7)
  box2 = ax2.get_position()
  ax2.set_position([box2.x0, box2.y0, box2.width * 0.8, box2.height*0.95])


  #Plot Data
  signado_contour = ax2.contourf(sigprob.longitude, sigprob.latitude, sigprob.values, levels = siglevels, colors = sigcolor_list)

  #Create Legend
  ax2.legend(slegend_patches, slegend_labels,  loc='upper left', fontsize=12, facecolor='white', markerscale=12,
                labelcolor='k', title='  Risk', title_fontsize=12, edgecolor='black', framealpha=1,bbox_to_anchor=(1.10, 1), fancybox=True, shadow=True,)

  #Add Counties and Coastlines
  ax2.coastlines(color='black',resolution='10m',zorder=99)
  ax2.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax2.add_feature(USCOUNTIES, facecolor='None', edgecolor='darkgray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax2.add_feature(lake)

    #Add Gridlines
  gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl2 = ax2.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)

  gl.x_labels_top = False
  gl.x_labels_bottom = True

  gl2.x_labels_top = False
  gl2.x_labels_bottom = True

  #Save Image
  if type_spcabs == 'SPC Like':
    plt.savefig(f'Nadocast_Day{period}_{product}_{year}-{month}-{day}-{time}UTC.png', bbox_inches = 'tight')
  elif type_spcabs == 'Absolutey Calibrated':
    plt.savefig(f'Nadocast_Day{period}_{product}_Abs_Calib_{year}-{month}-{day}-{time}UTC.png', bbox_inches = 'tight')

elif (product == 'tornado_all' or 'wind_all' or 'hail_all') and (time == '14') and (product !='all_hazards'):
  warnings.warn('Significant Hazard Forecasts Not Available At 14z')

  #Set Up Figure
  fig = plt.figure(figsize=(12, 12), facecolor='white', edgecolor="black", linewidth =6)
  ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
  ax.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax.set_box_aspect(0.7)
  box1 = ax.get_position()
  ax.set_position([box1.x0, box1.y0, box1.width * 0.8, box1.height*0.95])

  #Plot Data
  nado_contour = ax.contourf(prob.longitude, prob.latitude, prob.values, levels = levels, colors = color_list)
  if product == 'tornado' or product == 'hail' or product == 'wind':
    sig_contour = ax.contourf(sigprob.longitude, sigprob.latitude, sigprob.values, levels = siglevel, colors = sig_color, hatches = sig_hatch)
    sig_line = ax.contour(sigprob.longitude, sigprob.latitude, sigprob.values, levels = siglevel, colors = 'black', linewidths = 1)

  #Create Legend
  ax.legend(legend_patches, legend_labels,  loc='upper left', fontsize=12, facecolor='white', markerscale=12,
                labelcolor='k', title='  Risk', title_fontsize=12, edgecolor='black', framealpha=1,bbox_to_anchor=(1.10, 1), fancybox=True, shadow=True,)

  #Add Counties and Coastlines
  ax.coastlines(color='black',resolution='10m',zorder=99)
  ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='darkgray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax.add_feature(lake)

  #Add Figure Text
  plt.figtext(0.11, 0.23, f'SOURCE:Nadocast', weight='bold', color='black', fontsize=13, ha='left', alpha=0.8)

  if type_spcabs == 'SPC Like':
    if period == '1':
      plt.figtext(0.45, 0.73, f'Nadocast Day 1 {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)
    if period == '2':
      plt.figtext(0.45, 0.73, f'Nadocast Day 2 {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)
  elif type_spcabs == 'Absolutey Calibrated':
    if period == '1':
      plt.figtext(0.45, 0.73, f'Nadocast Day 1 Absolutely Calibrated {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)
    if period == '2':
      plt.figtext(0.45, 0.73, f'Nadocast Day 2 Absolutely Calibrated {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)

  #Add Gridlines
  gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl.x_labels_top = False
  gl.x_labels_bottom = True

  #Save Image
  if type_spcabs == 'SPC Like':
    plt.savefig(f'Nadocast_Day{period}_{product}_{year}-{month}-{day}-{time}UTC.png', bbox_inches = 'tight')
  elif type_spcabs == 'Absolutey Calibrated':
    plt.savefig(f'Nadocast_Day{period}_{product}_Abs_Calib_{year}-{month}-{day}-{time}UTC.png', bbox_inches = 'tight')

if (product == 'all_hazards') and (time != '14'):
  #Set Up Figure
  fig = plt.figure(figsize=(18, 12), facecolor='white', edgecolor="black", linewidth =6)

  ax=fig.add_subplot(2,3,1, projection=ccrs.PlateCarree())
  ax.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax.set_box_aspect(0.7)
  box1 = ax.get_position()
  ax.set_position([box1.x0, box1.y0, box1.width * 0.8, box1.height*0.95])

  #Plot Data
  tnado_contour = ax.contourf(torprob.longitude, torprob.latitude, torprob.values, levels = tproblevels, colors = tprobcolor_list)
  tsig_contour = ax.contourf(storprob.longitude, storprob.latitude, storprob.values, levels = tprobsiglevel, colors = tprobsig_color, hatches = tprobsig_hatch)
  tsig_line = ax.contour(storprob.longitude, storprob.latitude, storprob.values, levels = tprobsiglevel, colors = 'black', linewidths = 1)

  #Create Legend
  ax.legend(tproblegend_patches, tproblegend_labels,  loc='upper left', fontsize=12, facecolor='white', markerscale=12,
                labelcolor='k', title='  Risk', title_fontsize=12, edgecolor='black', framealpha=1,bbox_to_anchor=(1, 1), fancybox=True, shadow=True,)

  #Add Counties and Coastlines
  ax.coastlines(color='black',resolution='10m',zorder=99)
  ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='darkgray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax.add_feature(lake)


  ax2=fig.add_subplot(2,3,2, projection=ccrs.PlateCarree())
  ax2.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax2.set_box_aspect(0.7)
  box2 = ax2.get_position()
  ax2.set_position([box2.x0, box2.y0, box2.width * 0.8, box2.height*0.95])


  #Plot Data
  wnado_contour = ax2.contourf(windprob.longitude, windprob.latitude, windprob.values, levels = wproblevels, colors = wprobcolor_list)
  wsig_contour = ax2.contourf(swindprob.longitude, swindprob.latitude, swindprob.values, levels = wprobsiglevel, colors = wprobsig_color, hatches = wprobsig_hatch)
  wsig_line = ax2.contour(swindprob.longitude, swindprob.latitude, swindprob.values, levels = wprobsiglevel, colors = 'black', linewidths = 1)

  #Create Legend
  ax2.legend(wproblegend_patches, wproblegend_labels,  loc='upper left', fontsize=12, facecolor='white', markerscale=12,
                labelcolor='k', title='  Risk', title_fontsize=12, edgecolor='black', framealpha=1,bbox_to_anchor=(1, 1), fancybox=True, shadow=True,)

  #Add Counties and Coastlines
  ax2.coastlines(color='black',resolution='10m',zorder=99)
  ax2.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax2.add_feature(USCOUNTIES, facecolor='None', edgecolor='darkgray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax2.add_feature(lake)


  ax3=fig.add_subplot(2,3,3, projection=ccrs.PlateCarree())
  ax3.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax3.set_box_aspect(0.7)
  box3 = ax3.get_position()
  ax3.set_position([box3.x0, box3.y0, box3.width * 0.8, box3.height*0.95])


  #Plot Data
  hnado_contour = ax3.contourf(hailprob.longitude, hailprob.latitude, hailprob.values, levels = hproblevels, colors = hprobcolor_list)
  hsig_contour = ax3.contourf(shailprob.longitude, shailprob.latitude, shailprob.values, levels = hprobsiglevel, colors = hprobsig_color, hatches = hprobsig_hatch)
  hsig_line = ax3.contour(shailprob.longitude, shailprob.latitude, shailprob.values, levels = hprobsiglevel, colors = 'black', linewidths = 1)

  #Create Legend
  ax3.legend(hproblegend_patches, hproblegend_labels,  loc='upper left', fontsize=12, facecolor='white', markerscale=12,
                labelcolor='k', title='  Risk', title_fontsize=12, edgecolor='black', framealpha=1,bbox_to_anchor=(1, 1), fancybox=True, shadow=True,)

  #Add Counties and Coastlines
  ax3.coastlines(color='black',resolution='10m',zorder=99)
  ax3.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax3.add_feature(USCOUNTIES, facecolor='None', edgecolor='darkgray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax3.add_feature(lake)

  ax4=fig.add_subplot(2,3,4, projection=ccrs.PlateCarree())
  ax4.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax4.set_box_aspect(0.7)
  box4 = ax4.get_position()
  ax4.set_position([box4.x0, box4.y0, box4.width * 0.8, box4.height*0.95])


  #Plot Data
  signado_contour = ax4.contourf(storprob.longitude, storprob.latitude, storprob.values, levels = tsiglevels, colors = tsigcolor_list)

  #Create Legend
  ax4.legend(stlegend_patches, stlegend_labels,  loc='upper left', fontsize=12, facecolor='white', markerscale=12,
                labelcolor='k', title='  Risk', title_fontsize=12, edgecolor='black', framealpha=1,bbox_to_anchor=(1, 1), fancybox=True, shadow=True,)

  #Add Counties and Coastlines
  ax4.coastlines(color='black',resolution='10m',zorder=99)
  ax4.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax4.add_feature(USCOUNTIES, facecolor='None', edgecolor='darkgray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax4.add_feature(lake)


  ax5=fig.add_subplot(2,3,5, projection=ccrs.PlateCarree())
  ax5.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax5.set_box_aspect(0.7)
  box5 = ax5.get_position()
  ax5.set_position([box5.x0, box5.y0, box5.width * 0.8, box5.height*0.95])


  #Plot Data
  sigwind_contour = ax5.contourf(swindprob.longitude, swindprob.latitude, swindprob.values, levels = wsiglevels, colors = wsigcolor_list)

  #Create Legend
  ax5.legend(wSlegend_patches, wSlegend_labels,  loc='upper left', fontsize=12, facecolor='white', markerscale=12,
                labelcolor='k', title='  Risk', title_fontsize=12, edgecolor='black', framealpha=1,bbox_to_anchor=(1, 1), fancybox=True, shadow=True,)

  #Add Counties and Coastlines
  ax5.coastlines(color='black',resolution='10m',zorder=99)
  ax5.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax5.add_feature(USCOUNTIES, facecolor='None', edgecolor='darkgray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax5.add_feature(lake)

  ax6=fig.add_subplot(2,3,6, projection=ccrs.PlateCarree())
  ax6.set_extent((WestLong, EastLong, SouthLat, NorthLat))
  ax6.set_box_aspect(0.7)
  box6 = ax6.get_position()
  ax6.set_position([box6.x0, box6.y0, box6.width * 0.8, box6.height*0.95])

  #Plot Data
  sighail_contour = ax6.contourf(shailprob.longitude, shailprob.latitude, shailprob.values, levels = hsiglevels, colors = hsigcolor_list)

  #Create Legend
  ax6.legend(hSlegend_patches, hSlegend_labels,  loc='upper left', fontsize=12, facecolor='white', markerscale=12,
                labelcolor='k', title='  Risk', title_fontsize=12, edgecolor='black', framealpha=1,bbox_to_anchor=(1, 1), fancybox=True, shadow=True,)

  #Add Counties and Coastlines
  ax6.coastlines(color='black',resolution='10m',zorder=99)
  ax6.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
  ax6.add_feature(USCOUNTIES, facecolor='None', edgecolor='darkgray', zorder=9)
  lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
  ax6.add_feature(lake)

  #Add Gridlines
  gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl2 = ax2.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl3= ax3.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl4 = ax4.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl5 = ax5.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
  gl6 = ax6.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)

  gl.x_labels_top = False
  gl.x_labels_bottom = True

  gl2.x_labels_top = False
  gl2.x_labels_bottom = True

  gl3.x_labels_top = False
  gl3.x_labels_bottom = True

  gl4.x_labels_top = False
  gl4.x_labels_bottom = True

  gl5.x_labels_top = False
  gl5.x_labels_bottom = True

  gl6.x_labels_top = False
  gl6.x_labels_bottom = True


  #Add Figure Text
  plt.figtext(0.11, 0.10, f'SOURCE:Nadocast', weight='bold', color='black', fontsize=13, ha='left', alpha=0.8)

  if type_spcabs == 'SPC Like':
    if period == '1':
      plt.figtext(0.5, 0.88, f'Nadocast Day 1 Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=20)
    if period == '2':
      plt.figtext(0.5, 0.88, f'Nadocast Day 2 Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=20)
  elif type_spcabs == 'Absolutey Calibrated':
    if period == '1':
      plt.figtext(0.5, 0.88, f'Nadocast Day 1 Absolutely Calibrated Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=20)
    if period == '2':
      plt.figtext(0.5, 0.88, f'Nadocast Day 2 Absolutely Calibrated Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=20)

  plt.figtext(0.14, 0.83, f'Overall Tornado Risk', ha='left', weight='bold', fontsize=16)
  plt.figtext(0.14, 0.43, f'Significant Tornado Risk', ha='left', weight='bold', fontsize=16)

  plt.figtext(0.41, 0.83, f'Overall Wind Risk', ha='left', weight='bold', fontsize=16)
  plt.figtext(0.41, 0.43, f'Significant Wind Risk', ha='left', weight='bold', fontsize=16)

  plt.figtext(0.68, 0.83, f'Overall Hail Risk', ha='left', weight='bold', fontsize=16)
  plt.figtext(0.68, 0.43, f'Significant Hail Risk', ha='left', weight='bold', fontsize=16)

  #Save Image
  if type_spcabs == 'SPC Like':
    plt.savefig(f'Nadocast_Day{period}_{product}_{year}-{month}-{day}-{time}UTC.png', bbox_inches = 'tight')
  elif type_spcabs == 'Absolutey Calibrated':
    plt.savefig(f'Nadocast_Day{period}_{product}_Abs_Calib_{year}-{month}-{day}-{time}UTC.png', bbox_inches = 'tight')